In [52]:
import os
from nltk.corpus import stopwords
import re 
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np

def remove_stopword(x, lista_stopwords):
    return [y for y in x if y not in lista_stopwords]

def clean_text(text):
    '''Make text lowercase, remove text in square brackets,remove links,remove punctuation
    and remove words containing numbers. Also, we added the unicode line for accent marks'''
    text = str(text).lower()
    text = re.sub('\[.*?\]', '', text) #Punctuations...
    text = re.sub('https?://\S+|www\.\S+', '', text)
    text = re.sub('<.*?>+', '', text)
    #text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\n', '', text)
    text = re.sub('\w*\d\w*', '', text)
    #text = unidecode.unidecode(text)
    return text

def make_clean_dataframe(stopwords_espaniol, path_textos):
    #Accedo al path y jalo toda la info
    dicc={}
    for nombre_doc in os.listdir(path_textos):
        text_string = open(path_textos+'/'+nombre_doc).read()
        dicc[nombre_doc[:-4]] = text_string
    
    #Limpio y transformo el texto.
    dataframe = pd.DataFrame(dicc,index=[0]).T.rename(columns={0:'texto'}).reset_index()
    dataframe['temp_list'] = dataframe['texto'].apply(lambda x: clean_text(x))
    dataframe['temp_list'] = dataframe['temp_list'].apply(lambda x: str(x).split())
    dataframe['texto_limpio'] = dataframe['temp_list'].apply(lambda x: remove_stopword(x, stopwords_espaniol))
    dataframe = dataframe.rename(columns={'index':'nombre_doc'})

    for k,v in dataframe['texto_limpio'].items():
        dataframe.loc[k,'raw_clean_text'] = ' '.join(dataframe.loc[k,'texto_limpio'])
    
    return dataframe

agg_stopword = ['s', '2018','31','diciembre','financieros','000','2019','nota','grupo','valor','2017','resultados','compania','1',
 'total','consolidados','consolidado','razonable','gerencia','ciento','c','activos','cuentas','neto','us','efectivo','fecha','peru',
 'inretail','2','3','importe', 'aproximadamente','b','respectivamente','ver','ano','si','vida','anos','4','d','5','i','www','com',
 'aa', 'aaa', 'aaahipotecario', 'aaatat', 'aamnto', 'ab','ir','email','mes','niif','fmiv','bbb','ok','mzo','inc','alicorp','notas','dic']

import nltk
nltk.download('stopwords')

stopwords_espaniol = stopwords.words('spanish')
stopwords_espaniol.extend(agg_stopword)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\pablo\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [53]:
dataframe = make_clean_dataframe(stopwords_espaniol, 'data')
dataframe

,nombre_doc,texto,temp_list,texto_limpio,raw_clean_text
0,NOTAS_ALICORP_2018_1Q,alicorp s a a notas a los estados financieros ...,"[alicorp, s, a, a, notas, a, los, estados, fin...","[marzo, expresados, miles, soles, principios, ...",marzo expresados miles soles principios practi...
1,NOTAS_ALICORP_2018_2Q,alicorp s a a notas a los estados financieros ...,"[alicorp, s, a, a, notas, a, los, estados, fin...","[junio, expresados, miles, soles, principios, ...",junio expresados miles soles principios practi...
2,NOTAS_ALICORP_2018_3Q,alicorp s a a notas a los estados financieros ...,"[alicorp, s, a, a, notas, a, los, estados, fin...","[setiembre, expresados, miles, soles, principi...",setiembre expresados miles soles principios pr...
3,NOTAS_ALICORP_2018_4Q,alicorp s a a notas a los estados financieros ...,"[alicorp, s, a, a, notas, a, los, estados, fin...","[expresados, miles, soles, principios, practic...",expresados miles soles principios practicas co...
4,NOTAS_ALICORP_2019_1Q,alicorp s a a notas a los estados financieros ...,"[alicorp, s, a, a, notas, a, los, estados, fin...","[marzo, expresados, miles, soles, principios, ...",marzo expresados miles soles principios practi...
5,NOTAS_ALICORP_2019_2Q,alicorp s a a notas a los estados financieros ...,"[alicorp, s, a, a, notas, a, los, estados, fin...","[separados, junio, expresados, miles, soles, p...",separados junio expresados miles soles princip...
6,NOTAS_ALICORP_2019_3Q,alicorp s a a notas a los estados financieros ...,"[alicorp, s, a, a, notas, a, los, estados, fin...","[separados, septiembre, expresados, miles, sol...",separados septiembre expresados miles soles pr...
7,NOTAS_ALICORP_2019_4Q,alicorp s a a y subsidiarias notas a los estad...,"[alicorp, s, a, a, y, subsidiarias, notas, a, ...","[subsidiarias, expresados, miles, soles, princ...",subsidiarias expresados miles soles principios...
8,NOTAS_ALICORP_2020_1Q,alicorp s a a y subsidiarias notas a los estad...,"[alicorp, s, a, a, y, subsidiarias, notas, a, ...","[subsidiarias, marzo, expresados, miles, soles...",subsidiarias marzo expresados miles soles acti...


#### Hasta acá tenemos 9 estados financieros de una misma empresa ya limpios de caracteres y palabras basura. Ahora tenemos que Lemmatizar las palabras, es decir, volverlas a su raíz para mejor procesamiento. Luego, intentaremos clasificar los documentos según el modelo No Supervisado: Latent Dirichlet Allocation.

Recuerda en que la diferencia entre Lemmatizer y Stemmer yace en la metodología de la reducción de palabras. Mientras Lemmatizer se basa en un análisis morfológico de las palabras y requiere de un diccionario de especificación, Stemmer se basa en cortar prefijos y sufijos comunes a las palabras involucradas en el texto. 

In [54]:
from nltk.stem import PorterStemmer

def porter_stemmer(word):
    stemmer = PorterStemmer()
    return stemmer.stem(word)

In [55]:
dataframe['raw_clean_text'].str.split()

0    [marzo, expresados, miles, soles, principios, ...
1    [junio, expresados, miles, soles, principios, ...
2    [setiembre, expresados, miles, soles, principi...
3    [expresados, miles, soles, principios, practic...
4    [marzo, expresados, miles, soles, principios, ...
5    [separados, junio, expresados, miles, soles, p...
6    [separados, septiembre, expresados, miles, sol...
7    [subsidiarias, expresados, miles, soles, princ...
8    [subsidiarias, marzo, expresados, miles, soles...
Name: raw_clean_text, dtype: object

In [56]:
porter_stemmer('subsidiarias')

'subsidiaria'

In [57]:
dict_data = {k:v.split() for k, v in zip(dataframe['nombre_doc'],dataframe['raw_clean_text'])}

In [58]:
doc_clean = [*dataframe['raw_clean_text'].str.split()]

In [67]:
import gensim
from gensim import corpora
from gensim.models import CoherenceModel

#Debemos crear un diccionario de nuestro corpus (lista de docs), donde cada término único sea asignado a un index. 

#Convirtiendo la lista de documentos corpus en una Matriz de términos de documentos, usando el diccionario dict_data

dictionary = corpora.Dictionary(doc_clean)
doc_term_matrix = [dictionary.doc2bow(doc) for doc in doc_clean]

In [68]:
dictionary = corpora.Dictionary(doc_clean)

Ahora correremos el modelo LDA que gensim nos provee

In [69]:
LDA = gensim.models.ldamodel.LdaModel

ldamodel = LDA(doc_term_matrix, num_topics=5, id2word=dictionary, passes=40,random_state=300)

In [81]:
print(ldamodel.print_topics(num_topics=5, num_words=6))

[(0, '0.012*"arrendamiento" + 0.011*"activo" + 0.011*"inversiones" + 0.010*"arrendamientos" + 0.010*"pasivo" + 0.010*"derecho"'), (1, '0.017*"arrendamiento" + 0.013*"bonos" + 0.013*"emision" + 0.009*"subsidiarias" + 0.008*"vigentes" + 0.008*"bolivianos"'), (2, '0.011*"instrumentos" + 0.010*"inversiones" + 0.009*"ingresos" + 0.009*"contables" + 0.009*"tiempo" + 0.009*"cobertura"'), (3, '0.001*"inversiones" + 0.001*"emision" + 0.001*"marzo" + 0.001*"enero" + 0.001*"serie" + 0.001*"contables"'), (4, '0.001*"contables" + 0.001*"emision" + 0.001*"informacion" + 0.001*"inversiones" + 0.001*"serie" + 0.001*"gastos"')]


#### Métricas del modelo: 

Perplexity: Qué tan bueno es el modelo con respecto a palabras buenas.

Coherence Score: Qué tanto se sustenta el modelo con respecto a las palabras aledañas.

In [78]:
print('Perplexity: {}'.format(ldamodel.log_perplexity(doc_term_matrix))) #Mientras más bajo mejor.

#Cohere Score:
coherence_model_lda = CoherenceModel(model=ldamodel, texts= doc_clean, dictionary=dictionary, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()

print('Coherence Score: {}'.format(coherence_lda))

Perplexity: -6.5309116418709365
Coherence Score: 0.28504458324472415


In [73]:
ldamodel2 = LDA(doc_term_matrix, num_topics=3, id2word=dictionary, passes=60,random_state=300)
coherence_model_lda = CoherenceModel(model=ldamodel2, texts= doc_clean, dictionary=dictionary, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()

print('Coherence Score: {}'.format(coherence_lda))

Coherence Score: 0.2686555407995678


In [75]:
#Ahora le pondremos 600 passes
ldamodel2 = LDA(doc_term_matrix, num_topics=7, id2word=dictionary, passes=600,random_state=300)
coherence_model_lda = CoherenceModel(model=ldamodel2, texts= doc_clean, dictionary=dictionary, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()

print('Coherence Score: {}'.format(coherence_lda))

Coherence Score: 0.36828379358002655


In [79]:
print(ldamodel2.print_topics(num_topics=5, num_words=6))

[(6, '0.001*"construcciones" + 0.001*"mediante" + 0.001*"consorcio" + 0.001*"considera" + 0.001*"financiado" + 0.001*"computo"'), (0, '0.013*"arrendamiento" + 0.011*"activo" + 0.011*"inversiones" + 0.010*"arrendamientos" + 0.010*"pasivo" + 0.010*"derecho"'), (2, '0.011*"instrumentos" + 0.010*"inversiones" + 0.010*"ingresos" + 0.010*"contables" + 0.010*"tiempo" + 0.010*"cobertura"'), (5, '0.018*"arrendamiento" + 0.014*"bonos" + 0.014*"emision" + 0.010*"subsidiarias" + 0.009*"vigentes" + 0.009*"bolivianos"'), (3, '0.001*"construcciones" + 0.001*"mediante" + 0.001*"consorcio" + 0.001*"considera" + 0.001*"financiado" + 0.001*"computo"')]


Podemos ver que algunos de las palabras dentro de los tópicos nos pueden ser útiles. Pero, quedan las preguntas:

¿Cómo sé cuántos tópicos son óptimos dentro de mis datos? 

¿Cómo veo el puntaje de un tópico por documento?

#### Optimizando el Número de tópicos: 

Fuente: https://towardsdatascience.com/evaluate-topic-model-in-python-latent-dirichlet-allocation-lda-7d57484bb5d0

In [99]:
import numpy as np
import tqdm

def compute_coherence_values(corpus, dictionary, a, b):
    
    lda_model = gensim.models.LdaMulticore(corpus=corpus,
                                           id2word=dictionary,
                                           num_topics=10, 
                                           random_state=100,
                                           chunksize=100,
                                           passes=600,
                                           alpha=a,
                                           eta=b)
    
    coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=dictionary, coherence='c_v')
    
    return coherence_model_lda.get_coherence()

grid = {}
grid['Validation_Set'] = {}
# Rango de tópicos
min_topics = 2
max_topics = 11
step_size = 1
topics_range = range(min_topics, max_topics, step_size)
# Rango de optimización del alpha
alpha = list(np.arange(0.01, 1, 0.3))
alpha.append('symmetric')
alpha.append('asymmetric')
# Rango de optimización del beta
beta = list(np.arange(0.01, 1, 0.3))
beta.append('symmetric')
# VTest de validación
num_of_docs = len(doc_clean)
corpus_sets = [# gensim.utils.ClippedCorpus(corpus, num_of_docs*0.25), 
               # gensim.utils.ClippedCorpus(corpus, num_of_docs*0.5), 
               gensim.utils.ClippedCorpus(doc_clean, int(num_of_docs*0.75)), 
               doc_clean]
corpus_title = ['75% Corpus', '100% Corpus']
model_results = {'Validation_Set': [],
                 'Topics': [],
                 'Alpha': [],
                 'Beta': [],
                 'Coherence': []
                }
# Can take a long time to run
if 1 == 1:
    # iterate through validation corpuses
    for i in tqdm(range(len(corpus_sets))):
        # iterate through number of topics
        for k in topics_range:
            # iterate through alpha values
            for a in alpha:
                # iterare through beta values
                for b in beta:
                    # get the coherence score for the given parameters
                    cv = compute_coherence_values(corpus=corpus_sets[i], dictionary=dictionary, 
                                                  a=a, b=b)
                    # Save the model results
                    model_results['Validation_Set'].append(corpus_title[i])
                    model_results['Topics'].append(k)
                    model_results['Alpha'].append(a)
                    model_results['Beta'].append(b)
                    model_results['Coherence'].append(cv)
                    
                    pbar.update(1)
    pd.DataFrame(model_results).to_csv('lda_tuning_results.csv', index=False)
    pbar.close()














  0%|                                                                                          | 0/540 [00:00<?, ?it/s]

KeyboardInterrupt: 

### Fuentes a considerar:

https://www.youtube.com/watch?v=T05t-SqKArY

https://radimrehurek.com/gensim/models/ldamodel.html

https://towardsdatascience.com/end-to-end-topic-modeling-in-python-latent-dirichlet-allocation-lda-35ce4ed6b3e0

https://towardsdatascience.com/perplexity-intuition-and-derivation-105dd481c8f3

https://towardsdatascience.com/evaluate-topic-model-in-python-latent-dirichlet-allocation-lda-7d57484bb5d0

https://www.aclweb.org/anthology/D12-1087/

https://www.analyticsvidhya.com/blog/2016/08/beginners-guide-to-topic-modeling-in-python/

To-Supervised:

https://towardsdatascience.com/unsupervised-nlp-topic-models-as-a-supervised-learning-input-cf8ee9e5cf28